<a href="https://colab.research.google.com/github/Beyzanurekerr/food_food/blob/main/food_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gıda–Gıda Etkileşim Analizi (Food–Food Interaction Analysis)

Bu çalışmada, USDA FoodData Central ve FooDB kaynaklarından elde edilen veriler
kullanılarak gıdalar arasındaki ortak besin öğelerine dayalı etkileşimler analiz edilmiştir.

Amaç, ham verilerden anlamlı ve açıklanabilir bir karar destek sistemi üretmektir.


In [3]:
import pandas as pd
import numpy as np
import json
from collections import Counter


In [4]:
from google.colab import files

uploaded = files.upload()


Saving FoodData_Central_foundation_food_json_2025-12-18.json to FoodData_Central_foundation_food_json_2025-12-18.json


In [5]:
import json

path = "/content/FoodData_Central_foundation_food_json_2025-12-18.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

foods = data["FoundationFoods"]
len(foods)

365

In [6]:
food_rows = []

for food in foods:
    food_rows.append({
        "fdcId": food["fdcId"],
        "food_name": food["description"],
        "food_category": food.get("foodCategory", {}).get("description")
    })

food_df = pd.DataFrame(food_rows)



In [7]:
nutrient_rows = []

for food in foods:
    for n in food["foodNutrients"]:
        nutrient_rows.append({
            "fdcId": food["fdcId"],
            "food_name": food["description"],
            "nutrient_name": n["nutrient"]["name"],
            "unit": n["nutrient"]["unitName"],
            "amount": n.get("amount")
        })

nutrient_df = pd.DataFrame(nutrient_rows)



In [8]:
usda_full = nutrient_df.merge(food_df, on=["fdcId", "food_name"])
usda_full.head()


,fdcId,food_name,nutrient_name,unit,amount,food_category
0,321358,"Hummus, commercial","Cryptoxanthin, beta",µg,3.0,Legumes and Legume Products
1,321358,"Hummus, commercial",Lycopene,µg,0.0,Legumes and Legume Products
2,321358,"Hummus, commercial","Tocopherol, delta",mg,1.3,Legumes and Legume Products
3,321358,"Hummus, commercial","Tocotrienol, gamma",mg,0.0,Legumes and Legume Products
4,321358,"Hummus, commercial","Tocotrienol, delta",mg,0.0,Legumes and Legume Products


In [9]:
food_food = usda_full.merge(
    usda_full,
    on="nutrient_name",
    suffixes=("_A", "_B")
)

food_food = food_food[
    food_food["food_name_A"] != food_food["food_name_B"]
]


In [10]:
food_food_clean = food_food.loc[:, [
    'fdcId_A', 'food_name_A', 'food_category_A',
    'fdcId_B', 'food_name_B', 'food_category_B',
    'nutrient_name',
    'unit_A', 'amount_A', 'amount_B'
]]


In [11]:
food_food_clean['diff'] = (
    food_food_clean['amount_A'] - food_food_clean['amount_B']
).abs()


In [12]:
food_food_clean = food_food_clean[food_food_clean['diff'] > 5]


In [13]:
def interaction_level(diff):
    if diff > 100:
        return "High"
    elif diff > 20:
        return "Medium"
    else:
        return "Low"

food_food_clean['interaction_level'] = food_food_clean['diff'].apply(interaction_level)


In [14]:
nutrient_health = {
    "Cryptoxanthin, beta": "Antioxidant related health support",
    "Iron, Fe": "Anemia",
    "Sodium, Na": "Hypertension",
    "Sugars, Total": "Diabetes",
    "Fiber, total dietary": "Gut health"
}

food_food_clean['health_effect'] = food_food_clean['nutrient_name'].map(nutrient_health)


In [15]:
column_tr = {
    "fdcId_A": "Gıda_A_ID",
    "food_name_A": "Gıda_A_Adı",
    "food_category_A": "Gıda_A_Kategorisi",
    "fdcId_B": "Gıda_B_ID",
    "food_name_B": "Gıda_B_Adı",
    "food_category_B": "Gıda_B_Kategorisi",
    "nutrient_name": "Besin_Öğesi",
    "unit_A": "Birim",
    "amount_A": "Gıda_A_Miktar",
    "amount_B": "Gıda_B_Miktar",
    "diff": "Fark",
    "interaction_level": "Etkileşim_Seviyesi",
    "health_effect": "Sağlık_Etkisi"
}

nutrient_tr = {
    "Cryptoxanthin, beta": "Beta Kriptoksantin",
    "Iron, Fe": "Demir",
    "Sodium, Na": "Sodyum",
    "Vitamin C, total ascorbic acid": "C Vitamini",
    "Fiber, total dietary": "Diyet Lifi",
    "Sugars, Total": "Toplam Şeker"
}

category_tr = {
    "Legumes and Legume Products": "Baklagiller",
    "Vegetables and Vegetable Products": "Sebzeler",
    "Fruits and Fruit Juices": "Meyveler",
    "Nut and Seed Products": "Kuruyemişler",
    "Dairy and Egg Products": "Süt ve Yumurta Ürünleri",
    "Spices and Herbs": "Baharatlar"
}

interaction_tr = {
    "High": "Yüksek",
    "Medium": "Orta",
    "Low": "Düşük"
}

health_tr = {
    "Antioxidant related health support": "Antioksidan Destek",
    "Anemia": "Anemi",
    "Hypertension": "Hipertansiyon",
    "Diabetes": "Diyabet",
    "Gut health": "Bağırsak Sağlığı"
}

food_name_tr = {
    "Hummus, commercial": "Humus",
    "Nuts, almonds, dry roasted, with salt added": "Tuzlu Kavrulmuş Badem",
    "Kale, raw": "Kara Lahana (Çiğ)",
    "Kale, frozen, cooked, boiled, drained, without salt": "Kara Lahana (Haşlanmış)",
    "Peaches, yellow, raw": "Sarı Şeftali",
    "Mustard, prepared, yellow": "Hazır Hardal (Sarı)",
    "Tomatoes, grape, raw": "Çeri Domates",
    "Pickles, cucumber, dill or kosher dill": "Salatalık Turşusu",
    "Cheese, pasteurized process, American": "Amerikan Peyniri"
}



In [16]:
food_food_tr = food_food_clean.copy()

# Kolon adları
food_food_tr = food_food_tr.rename(columns=column_tr)

# 🟢 GIDA ADLARI TÜRKÇE
food_food_tr["Gıda_A_Adı"] = food_food_tr["Gıda_A_Adı"].map(
    food_name_tr
).fillna(food_food_tr["Gıda_A_Adı"])

food_food_tr["Gıda_B_Adı"] = food_food_tr["Gıda_B_Adı"].map(
    food_name_tr
).fillna(food_food_tr["Gıda_B_Adı"])

# Besin öğesi
food_food_tr["Besin_Öğesi"] = food_food_tr["Besin_Öğesi"].map(
    nutrient_tr
).fillna(food_food_tr["Besin_Öğesi"])

# Kategoriler
food_food_tr["Gıda_A_Kategorisi"] = food_food_tr["Gıda_A_Kategorisi"].map(
    category_tr
).fillna(food_food_tr["Gıda_A_Kategorisi"])

food_food_tr["Gıda_B_Kategorisi"] = food_food_tr["Gıda_B_Kategorisi"].map(
    category_tr
).fillna(food_food_tr["Gıda_B_Kategorisi"])

# Etkileşim seviyesi
food_food_tr["Etkileşim_Seviyesi"] = food_food_tr["Etkileşim_Seviyesi"].map(
    interaction_tr
)

# Sağlık etkisi
food_food_tr["Sağlık_Etkisi"] = food_food_tr["Sağlık_Etkisi"].map(
    health_tr
)


In [17]:
food_food_tr[[
    "Gıda_A_Adı",
    "Gıda_B_Adı",
    "Besin_Öğesi",
    "Fark",
    "Etkileşim_Seviyesi",
    "Sağlık_Etkisi"
]].head()


,Gıda_A_Adı,Gıda_B_Adı,Besin_Öğesi,Fark,Etkileşim_Seviyesi,Sağlık_Etkisi
2,Humus,Tuzlu Kavrulmuş Badem,Beta Kriptoksantin,6.0,Düşük,Antioksidan Destek
3,Humus,Kara Lahana (Çiğ),Beta Kriptoksantin,24.0,Orta,Antioksidan Destek
6,Humus,Sarı Şeftali,Beta Kriptoksantin,114.0,Yüksek,Antioksidan Destek
8,Humus,Kara Lahana (Haşlanmış),Beta Kriptoksantin,23.0,Orta,Antioksidan Destek
9,Humus,Hazır Hardal (Sarı),Beta Kriptoksantin,24.0,Orta,Antioksidan Destek


In [19]:
from google.colab import files

uploaded = files.upload()

Saving foodb.zip to foodb.zip


In [20]:
import zipfile

with zipfile.ZipFile("foodb.zip", "r") as zip_ref:
    zip_ref.extractall("/content/foodb")


In [24]:
import json
import pandas as pd

BASE_PATH = "/content/foodb/foodb_data"

# Function to safely load multiple JSON objects from a file
def load_json_data_from_file(filepath):
    data_list = []
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    try:
        # Try to load as a single JSON object/array
        parsed_data = json.loads(content)
        # If it's a single object, wrap it in a list to ensure consistent output
        if not isinstance(parsed_data, list):
            data_list.append(parsed_data)
        else:
            data_list = parsed_data
    except json.JSONDecodeError:
        # If it fails as a single object, try parsing line by line (JSON Lines format)
        lines = content.strip().split('\n')
        for line in lines:
            line = line.strip()
            if line:
                try:
                    data_list.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Warning: Could not decode JSON line from {filepath}: {line[:100]}... Error: {e}")
    return data_list

foodb_foods = load_json_data_from_file(f"{BASE_PATH}/Food.json")
foodb_compounds = load_json_data_from_file(f"{BASE_PATH}/Compound.json")
foodb_content = load_json_data_from_file(f"{BASE_PATH}/Content.json")
foodb_health = load_json_data_from_file(f"{BASE_PATH}/HealthEffect.json")

In [25]:
len(foodb_foods), len(foodb_compounds), len(foodb_content)


(992, 70477, 5691011)

In [27]:
nutrient_to_bioactive = {
    "Cryptoxanthin, beta": "Karotenoid",
    "Vitamin C, total ascorbic acid": "Antioksidan",
    "Iron, Fe": "Mineral",
    "Fiber, total dietary": "Prebiyotik",
    "Sugars, Total": "Metabolik"
}


In [28]:
bioactive_to_health = {
    "Karotenoid": "Antioksidan ve hücresel koruma",
    "Antioksidan": "Bağışıklık ve oksidatif stres",
    "Mineral": "Kan ve metabolik denge",
    "Prebiyotik": "Bağırsak sağlığı",
    "Metabolik": "Glisemik denge"
}


In [29]:
food_food_tr["Biyoaktif_Sınıf"] = food_food_tr["Besin_Öğesi"].map(
    nutrient_to_bioactive
)

food_food_tr["Biyoaktif_Etki"] = food_food_tr["Biyoaktif_Sınıf"].map(
    bioactive_to_health
)


In [30]:
nutrient_to_bioactive = {
    "Beta Kriptoksantin": "Karotenoid",
    "C Vitamini": "Antioksidan",
    "A Vitamini": "Retinoid",
    "E Vitamini": "Antioksidan",
    "Demir": "Mineral",
    "Sodyum": "Elektrolit",
    "Diyet Lifi": "Prebiyotik",
    "Toplam Şeker": "Metabolik"
}


In [31]:
bioactive_to_health = {
    "Karotenoid": "Antioksidan etki ve hücresel koruma sağlar",
    "Antioksidan": "Oksidatif stresin azaltılmasına katkı sağlar",
    "Retinoid": "Görme ve bağışıklık sistemi ile ilişkilidir",
    "Mineral": "Metabolik ve hematolojik dengeyi destekler",
    "Elektrolit": "Sıvı dengesi ve kan basıncı ile ilişkilidir",
    "Prebiyotik": "Bağırsak mikrobiyotasını destekler",
    "Metabolik": "Glisemik denge ile ilişkilidir"
}


In [32]:
food_food_tr["Biyoaktif_Sınıf"] = food_food_tr["Besin_Öğesi"].map(
    nutrient_to_bioactive
)

food_food_tr["Biyoaktif_Etki"] = food_food_tr["Biyoaktif_Sınıf"].map(
    bioactive_to_health
)


In [33]:
food_food_tr["Önleyici_Sağlık_Yorumu"] = (
    food_food_tr["Gıda_A_Adı"] + " ve " +
    food_food_tr["Gıda_B_Adı"] + " birlikte tüketildiğinde, " +
    food_food_tr["Besin_Öğesi"] + " üzerinden " +
    food_food_tr["Biyoaktif_Etki"] + "."
)


In [34]:
food_food_tr["Önleyici_Sağlık_Yorumu"] = (
    food_food_tr["Gıda_A_Adı"] + " ve " +
    food_food_tr["Gıda_B_Adı"] + " birlikte tüketildiğinde, " +
    food_food_tr["Besin_Öğesi"] + " üzerinden " +
    food_food_tr["Biyoaktif_Etki"] + "."
)



In [35]:
level_templates = {
    "Düşük": "besin katkısı dengelidir ve belirgin bir önleyici etki beklenmez",
    "Orta": "besin katkısı baskınlaşmaya başlamış olup düzenli tüketimde destekleyici olabilir",
    "Yüksek": "besin katkısı belirgin şekilde baskındır ve önleyici sağlık açısından anlamlı olabilir"
}


In [36]:
bioactive_focus = {
    "Karotenoid": "antioksidan ve hücresel koruma",
    "Antioksidan": "bağışıklık ve oksidatif stres",
    "Retinoid": "görme ve bağışıklık sistemi",
    "Mineral": "metabolik ve hematolojik denge",
    "Elektrolit": "sıvı dengesi ve kan basıncı",
    "Prebiyotik": "bağırsak mikrobiyotası",
    "Metabolik": "glisemik denge"
}

def onleyici_yorum(row):
    level_text = level_templates.get(row["Etkileşim_Seviyesi"], "")
    focus = bioactive_focus.get(row["Biyoaktif_Sınıf"], "genel sağlık")

    return (
        f"{row['Gıda_A_Adı']} ve {row['Gıda_B_Adı']} birlikte tüketildiğinde, "
        f"{row['Besin_Öğesi']} açısından {row['Etkileşim_Seviyesi'].lower()} düzeyde "
        f"bir etkileşim görülür; bu durum {focus} açısından {level_text}."
    )

In [37]:
food_food_tr["Önleyici_Sağlık_Yorumu"] = food_food_tr.apply(onleyici_yorum, axis=1)


In [38]:
food_food_tr[[
    "Gıda_A_Adı",
    "Gıda_B_Adı",
    "Besin_Öğesi",
    "Etkileşim_Seviyesi",
    "Biyoaktif_Sınıf",
    "Önleyici_Sağlık_Yorumu"
]].head()


,Gıda_A_Adı,Gıda_B_Adı,Besin_Öğesi,Etkileşim_Seviyesi,Biyoaktif_Sınıf,Önleyici_Sağlık_Yorumu
2,Humus,Tuzlu Kavrulmuş Badem,Beta Kriptoksantin,Düşük,Karotenoid,Humus ve Tuzlu Kavrulmuş Badem birlikte tüketi...
3,Humus,Kara Lahana (Çiğ),Beta Kriptoksantin,Orta,Karotenoid,Humus ve Kara Lahana (Çiğ) birlikte tüketildiğ...
6,Humus,Sarı Şeftali,Beta Kriptoksantin,Yüksek,Karotenoid,"Humus ve Sarı Şeftali birlikte tüketildiğinde,..."
8,Humus,Kara Lahana (Haşlanmış),Beta Kriptoksantin,Orta,Karotenoid,Humus ve Kara Lahana (Haşlanmış) birlikte tüke...
9,Humus,Hazır Hardal (Sarı),Beta Kriptoksantin,Orta,Karotenoid,Humus ve Hazır Hardal (Sarı) birlikte tüketild...


In [39]:
base_food_tr = {
    "almond": "Badem",
    "apple": "Elma",
    "applesauce": "Elma Püresi",
    "apricot": "Kayısı",
    "arugula": "Roka",
    "asparagus": "Kuşkonmaz",
    "avocado": "Avokado",
    "banana": "Muz",
    "bean": "Fasulye",
    "beef": "Dana Eti",
    "beet": "Pancar",
    "bison": "Bizon Eti",
    "blackberry": "Böğürtlen",
    "blueberry": "Yaban Mersini",
    "bread": "Ekmek",
    "broccoli": "Brokoli",
    "butter": "Tereyağı",
    "carrot": "Havuç",
    "cheese": "Peynir",
    "chicken": "Tavuk",
    "chickpeas": "Nohut",
    "cod": "Morina Balığı",
    "cucumber": "Salatalık",
    "egg": "Yumurta",
    "eggplant": "Patlıcan",
    "fig": "İncir",
    "fish": "Balık",
    "flour": "Un",
    "garlic": "Sarımsak",
    "grape": "Üzüm",
    "grapefruit": "Greyfurt",
    "ham": "Jambon",
    "kale": "Kara Lahana",
    "lentils": "Mercimek",
    "lettuce": "Marul",
    "milk": "Süt",
    "mushroom": "Mantar",
    "nuts": "Kuruyemiş",
    "oats": "Yulaf",
    "oil": "Yağ",
    "onion": "Soğan",
    "orange": "Portakal",
    "peas": "Bezelye",
    "pepper": "Biber",
    "pineapple": "Ananas",
    "pork": "Domuz Eti",
    "potatoes": "Patates",
    "rice": "Pirinç",
    "salmon": "Somon",
    "spinach": "Ispanak",
    "squash": "Kabak",
    "strawberries": "Çilek",
    "tomato": "Domates",
    "tuna": "Ton Balığı",
    "turkey": "Hindi",
    "yogurt": "Yoğurt"
}


In [40]:
descriptor_tr = {
    "raw": "çiğ",
    "canned": "konserve",
    "unsweetened": "şekersiz",
    "whole milk": "tam yağlı",
    "low fat": "az yağlı",
    "nonfat": "yağsız",
    "frozen": "dondurulmuş",
    "dried": "kurutulmuş",
    "with skin": "kabuklu",
    "peeled": "soyulmuş",
    "seedless": "çekirdeksiz",
    "shelf stable": "raf ömrü uzun",
    "from concentrate": "konsantreden"
}


In [41]:
def translate_food(food):
    food_l = food.lower()

    main = None
    for k, v in base_food_tr.items():
        if k in food_l:
            main = v
            break

    if main is None:
        main = food.split(",")[0]

    desc = []
    for k, v in descriptor_tr.items():
        if k in food_l:
            desc.append(v)

    if desc:
        return f"{main} ({', '.join(desc)})"
    else:
        return main


In [44]:
all_foods = usda_full['food_name'].unique()

food_tr_map = {food: translate_food(food) for food in all_foods}

foods_tr_df = pd.DataFrame({
    "Food_EN": all_foods,
    "Food_TR": [food_tr_map[f] for f in all_foods]
})

foods_tr_df

,Food_EN,Food_TR
0,"Hummus, commercial",Hummus
1,"Tomatoes, grape, raw",Üzüm (çiğ)
2,"Beans, snap, green, canned, regular pack, drai...",Fasulye (konserve)
3,"Frankfurter, beef, unheated",Dana Eti
4,"Nuts, almonds, dry roasted, with salt added",Badem
...,...,...
360,"Swordfish, frozen, wild caught",Balık (dondurulmuş)
361,"Tuna, ahi or yellowfin, frozen, wild caught",Ton Balığı (dondurulmuş)
362,"Turnips, raw",Turnips (çiğ)
363,"Watermelon, seedless, flesh only, raw","Watermelon (çiğ, çekirdeksiz)"


In [45]:
foods_tr_df.to_json("all_foods_tr.json", orient='records', indent=2, force_ascii=False)

In [46]:
import json
import itertools

def load_data(json_path: str):
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)


def normalize(text: str) -> str:
    return text.lower().strip()


def match_foods(user_foods, all_foods_in_data):
    matched = []
    for food_item_in_data in all_foods_in_data:
        for u in user_foods:
            if normalize(u) in normalize(food_item_in_data):
                matched.append(food_item_in_data)
    return list(set(matched))


def get_food_interactions(user_foods, data):
    """
    user_foods: Kullanıcının girdiği besin listesi (örn: ["Elma", "Yoğurt", "Mantar"])
    data: Önceden oluşturulmuş gıda etkileşim JSON verisi
    """
    # Assuming data["matched_foods"] is a list of strings
    matched_foods = match_foods(
        user_foods,
        data["matched_foods"]
    )

    interactions = []
    for item in data.get("interactions", []):
        if (
            item["food_1"] in matched_foods or
            item["food_2"] in matched_foods
        ):
            interactions.append(item)

    non_interactions = []
    # Use 'non_interacting_pairs' from 'output' and assume items are strings
    for item in data.get("non_interacting_pairs", []):
        if item in matched_foods:
            non_interactions.append(item)

    return {
        "query_foods": user_foods,
        "matched_foods": matched_foods,
        "interactions": interactions,
        "non_interactions": non_interactions
    }

In [48]:
import json
import pandas as pd

# 1. Get all unique food names for 'matched_foods'
all_unique_food_names = pd.concat([food_food_tr['Gıda_A_Adı'], food_food_tr['Gıda_B_Adı']]).unique().tolist()

# 2. Prepare 'interactions' list from food_food_tr
interactions_list = []
for index, row in food_food_tr.iterrows():
    interactions_list.append({
        "food_1": row["Gıda_A_Adı"],
        "food_2": row["Gıda_B_Adı"],
        "nutrient_name": row["Besin_Öğesi"],
        "interaction_level": row["Etkileşim_Seviyesi"],
        "health_effect": row["Sağlık_Etkisi"],
        "bioactive_class": row["Biyoaktif_Sınıf"],
        "preventive_health_comment": row["Önleyici_Sağlık_Yorumu"]
    })

# 3. Create the 'output' dictionary
output = {
    "matched_foods": all_unique_food_names,
    "interactions": interactions_list,
    "non_interacting_pairs": [] # This is an assumption as food_food_tr is already filtered
}

with open("all_foods_match_status.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("JSON başarıyla kaydedildi.")

JSON başarıyla kaydedildi.


In [49]:
!ls *.json


all_foods_match_status.json
all_foods_tr.json
FoodData_Central_foundation_food_json_2025-12-18.json
